In [ ]:
# iorfa_l1_flow_indicators_fast.py
from __future__ import annotations

from collections import namedtuple
from dataclasses import dataclass
from typing import Literal

import numpy as np
import gurobipy as gp
from gurobipy import GRB
from sklearn import tree


class OptimalRegressionTreeFlow:
    """
    Optimal regression tree with integrated linear term, L1 loss.

    Prediction:
        f(x) = x @ beta + gamma_leaf(x)

    New (optional) speed/accuracy features while keeping interpretability:
      - routing_formulation:
            "indicator" (your original indicator routing), or
            "bigm"      (no routing indicators, no rdir binaries; typically faster)
      - leaf_value_formulation:
            "indicator" (your original leaf-value indicators), or
            "bigm"      (replaces with tight big-M constraints; typically faster)
      - max_thresholds:
            if set, restrict b[t] to a small set of candidate thresholds per feature
            (interpretability preserved; often faster and more stable)
      - lambda_beta_l1 / lambda_gamma_l1:
            RuleFit-like sparsity on linear term and/or leaf offsets
      - polish:
            after MIP, fix integer vars and re-optimize continuous vars (LP), fast improvement

    Objective:
        min (1/n) * sum_i | y_i - x_i^T beta - s_i |  + alpha * sum_{branch t} d_t
            + lambda_beta_l1 * sum_j |beta_j| + lambda_gamma_l1 * sum_leaf |gamma_leaf|
    """

    def __init__(
        self,
        max_depth: int = 3,
        min_samples_split: int | None = None,   # alias
        min_samples_leaf: int | None = None,    # preferred
        alpha: float = 0.0,

        # speed/accuracy knobs
        routing_formulation: Literal["indicator", "bigm"] = "bigm",
        leaf_value_formulation: Literal["indicator", "bigm"] = "bigm",
        max_thresholds: int | None = None,  # e.g. 32; None keeps continuous b
        threshold_strategy: Literal["quantile", "uniform"] = "quantile",
        lambda_beta_l1: float = 0.0,
        lambda_gamma_l1: float = 0.0,
        polish: bool = True,
        polish_timelimit: float = 10.0,

        warmstart: bool = True,
        timelimit: float = 600.0,
        mipgap: float | None = None,
        output: bool = True,
        gamma_bounds: tuple[float, float] = (-10000.0, 10000.0),
        beta_bounds: tuple[float, float] | None = None,
        eps_tie: float = 1e-9,
        threads: int = 0,
        seed: int = 0,
        heuristics: float | None = None,
    ):
        self.max_depth = int(max_depth)

        if min_samples_leaf is not None:
            self.min_samples_leaf = int(min_samples_leaf)
        elif min_samples_split is not None:
            self.min_samples_leaf = int(min_samples_split)
        else:
            self.min_samples_leaf = 2

        self.alpha = float(alpha)
        self.routing_formulation = routing_formulation
        self.leaf_value_formulation = leaf_value_formulation
        self.max_thresholds = None if max_thresholds is None else int(max_thresholds)
        self.threshold_strategy = threshold_strategy
        self.lambda_beta_l1 = float(lambda_beta_l1)
        self.lambda_gamma_l1 = float(lambda_gamma_l1)
        self.polish = bool(polish)
        self.polish_timelimit = float(polish_timelimit)

        self.warmstart = bool(warmstart)
        self.timelimit = float(timelimit)
        self.mipgap = mipgap
        self.output = bool(output)
        self.gamma_bounds = (float(gamma_bounds[0]), float(gamma_bounds[1]))
        self.beta_bounds = beta_bounds
        self.eps_tie = float(eps_tie)
        self.threads = int(threads)
        self.seed = int(seed)
        self.heuristics = heuristics

        self.trained = False
        self.optgap = None

        self.feature_names_ = None
        self.feature_mins_ = None
        self.feature_maxs_ = None
        self._threshold_candidates_ = None  # if max_thresholds is used

        # learned params
        self._beta = None
        self._gamma = None
        self._a = None
        self._b = None
        self._d = None

        # node indices: 1..(2^(D+1)-1)
        self.n_index = [i + 1 for i in range(2 ** (self.max_depth + 1) - 1)]
        self.b_index = self.n_index[: -2**self.max_depth]         # branch nodes
        self.l_index = self.n_index[-2**self.max_depth :]         # leaves

        self.n = None
        self.p = None
        self.m = None

    # ---------------------------
    # Public API
    # ---------------------------
    def fit(self, x, y):
        if hasattr(x, "columns"):
            self.feature_names_ = list(x.columns)

        x = np.asarray(x, dtype=float)
        y = np.asarray(y, dtype=float).ravel()

        if x.ndim != 2:
            raise ValueError("x must be 2D (n_samples, n_features)")
        if y.ndim != 1:
            raise ValueError("y must be 1D (n_samples,)")
        if x.shape[0] != y.shape[0]:
            raise ValueError("x and y must have same number of rows")

        self.n, self.p = x.shape
        self.feature_mins_ = np.min(x, axis=0)
        self.feature_maxs_ = np.max(x, axis=0)

        if self.output:
            print(f"Training data include {self.n} instances, {self.p} features.")

        (
            m,
            a, b, d,
            q,
            l, N,
            beta, gamma,
            s,
            r, u,
            zsplit,  # optional split-choice vars if thresholds discretized
        ) = self._buildMILP(x, y)

        m.update()

        if self.warmstart:
            self._setStart(
                x, y,
                a, b, d,
                q,
                l, N,
                beta, gamma,
                s,
                r, u,
                zsplit,
            )

        m.optimize()

        # Optional fast "polish": fix binaries and re-optimize continuous vars as LP
        if self.polish and m.SolCount > 0:
            self._polish_solution(
                m,
                a=a, d=d, q=q, l=l, zsplit=zsplit,
            )

        self.m = m
        self.optgap = getattr(m, "MIPGap", None)

        self._a = {(j, t): a[j, t].X for j in range(self.p) for t in self.b_index}
        self._b = {t: b[t].X for t in self.b_index}
        self._d = {t: d[t].X for t in self.b_index}
        self._beta = np.array([beta[j].X for j in range(self.p)], dtype=float)
        self._gamma = {t: gamma[t].X for t in self.l_index}

        self.trained = True
        return self

    def predict(self, x):
        if not self.trained:
            raise AssertionError("This instance is not fitted yet.")

        x = np.asarray(x, dtype=float)
        if x.ndim == 1:
            x = x.reshape(1, -1)
        if x.shape[1] != self.p:
            raise ValueError(f"Expected {self.p} features, got {x.shape[1]}")

        y_pred = np.zeros(x.shape[0], dtype=float)

        for i, xi in enumerate(x):
            t = 1
            while t not in self.l_index:
                if self._d.get(t, 0.0) < 0.5:
                    t = 2 * t + 1
                    continue

                split_val = 0.0
                for j in range(self.p):
                    split_val += self._a.get((j, t), 0.0) * xi[j]

                if split_val + self.eps_tie >= self._b[t]:
                    t = 2 * t + 1
                else:
                    t = 2 * t
            y_pred[i] = float(np.dot(xi, self._beta) + self._gamma[t])

        return y_pred

    # ---------------------------
    # Core MIP
    # ---------------------------
    def _buildMILP(self, x: np.ndarray, y: np.ndarray):
        m = gp.Model("iorfa_l1_flow_fast")

        m.Params.OutputFlag = 1 if self.output else 0
        m.Params.LogToConsole = 1 if self.output else 0
        m.Params.TimeLimit = self.timelimit
        m.Params.Threads = self.threads
        m.ModelSense = GRB.MINIMIZE
        if self.heuristics is not None:
            m.Params.Heuristics = float(self.heuristics)
        m.Params.Seed = self.seed
        if self.mipgap is not None:
            m.Params.MIPGap = float(self.mipgap)

        # -----------------------
        # Variables
        # -----------------------
        a = m.addVars(self.p, self.b_index, vtype=GRB.BINARY, name="a")
        b = m.addVars(self.b_index, vtype=GRB.CONTINUOUS, name="b")
        d = m.addVars(self.b_index, vtype=GRB.BINARY, name="d")

        # flow: q[i,t] is 1 if sample i reaches node t
        q = m.addVars(self.n, self.n_index, vtype=GRB.BINARY, name="q")

        # leaf activation + counts
        l = m.addVars(self.l_index, vtype=GRB.BINARY, name="l")
        N = m.addVars(self.l_index, lb=0.0, vtype=GRB.CONTINUOUS, name="N")

        # linear term
        if self.beta_bounds is None:
            beta = m.addVars(self.p, vtype=GRB.CONTINUOUS, name="beta")
        else:
            lb, ub = self.beta_bounds
            beta = m.addVars(self.p, lb=float(lb), ub=float(ub), vtype=GRB.CONTINUOUS, name="beta")

        # leaf offsets
        Lower, Upper = self.gamma_bounds
        gamma = m.addVars(self.l_index, lb=Lower, ub=Upper, vtype=GRB.CONTINUOUS, name="gamma")

        # s_i = selected gamma
        s = m.addVars(self.n, vtype=GRB.CONTINUOUS, name="s")

        # residual + abs
        r = m.addVars(self.n, lb=-GRB.INFINITY, vtype=GRB.CONTINUOUS, name="r")
        u = m.addVars(self.n, lb=0.0, vtype=GRB.CONTINUOUS, name="abs_r")

        # Optional: discrete split choices (feature, threshold)
        zsplit = None
        if self.max_thresholds is not None:
            self._threshold_candidates_ = self._compute_threshold_candidates(x)
            keys = []
            for t in self.b_index:
                for j in range(self.p):
                    Kj = len(self._threshold_candidates_[j])
                    for k in range(Kj):
                        keys.append((j, k, t))
            zsplit = m.addVars(keys, vtype=GRB.BINARY, name="zsplit")

        # -----------------------
        # Objective
        # -----------------------
        complexity = gp.quicksum(d[t] for t in self.b_index)

        obj = (1.0 / self.n) * u.sum() + self.alpha * complexity

        # RuleFit-style sparsity penalties (optional)
        if self.lambda_beta_l1 > 0.0:
            beta_abs = m.addVars(self.p, lb=0.0, vtype=GRB.CONTINUOUS, name="beta_abs")
            m.addConstrs((beta_abs[j] >= beta[j] for j in range(self.p)), name="beta_abs_pos")
            m.addConstrs((beta_abs[j] >= -beta[j] for j in range(self.p)), name="beta_abs_neg")
            obj += self.lambda_beta_l1 * gp.quicksum(beta_abs[j] for j in range(self.p))

        if self.lambda_gamma_l1 > 0.0:
            gamma_abs = m.addVars(self.l_index, lb=0.0, vtype=GRB.CONTINUOUS, name="gamma_abs")
            m.addConstrs((gamma_abs[lf] >= gamma[lf] for lf in self.l_index), name="gamma_abs_pos")
            m.addConstrs((gamma_abs[lf] >= -gamma[lf] for lf in self.l_index), name="gamma_abs_neg")
            obj += self.lambda_gamma_l1 * gp.quicksum(gamma_abs[lf] for lf in self.l_index)

        m.setObjective(obj)

        # -----------------------
        # Structure / flow
        # -----------------------
        m.addConstrs((q[i, 1] == 1 for i in range(self.n)), name="root_flow")

        for t in self.b_index:
            lt = 2 * t
            rt = 2 * t + 1

            # child flows sum to parent
            m.addConstrs((q[i, lt] + q[i, rt] == q[i, t] for i in range(self.n)), name=f"flow[{t}]")

            # if inactive, force left=0 and right=parent (symmetry break)
            m.addConstrs((q[i, lt] <= d[t] for i in range(self.n)), name=f"no_left_if_inactive[{t}]")
            m.addConstrs((q[i, rt] >= q[i, t] - d[t] for i in range(self.n)), name=f"must_go_right_if_inactive[{t}]")

        # one leaf per sample
        m.addConstrs((gp.quicksum(q[i, lf] for lf in self.l_index) == 1 for i in range(self.n)), name="one_leaf")

        # leaf counts / activation
        m.addConstrs((N[lf] == gp.quicksum(q[i, lf] for i in range(self.n)) for lf in self.l_index), name="leaf_counts")
        m.addConstrs((q[i, lf] <= l[lf] for i in range(self.n) for lf in self.l_index), name="leaf_active_if_used")
        m.addConstrs((N[lf] >= self.min_samples_leaf * l[lf] for lf in self.l_index), name="min_leaf_size")

        # split selection + hierarchy
        m.addConstrs((d[t] <= d[t // 2] for t in self.b_index if t != 1), name="parent_active")

        if zsplit is None:
            # original: pick exactly one feature if split is active
            m.addConstrs((gp.quicksum(a[j, t] for j in range(self.p)) == d[t] for t in self.b_index), name="one_feat_if_split")

            # b bounds: if inactive, a=0 => b forced to 0 by these bounds
            feature_min = self.feature_mins_
            feature_max = self.feature_maxs_
            m.addConstrs(
                (b[t] <= gp.quicksum(feature_max[j] * a[j, t] for j in range(self.p)) for t in self.b_index),
                name="b_ub",
            )
            m.addConstrs(
                (b[t] >= gp.quicksum(feature_min[j] * a[j, t] for j in range(self.p)) for t in self.b_index),
                name="b_lb",
            )
        else:
            # Discrete (feature, threshold) choice:
            # sum_{j,k} zsplit[j,k,t] == d[t]
            for t in self.b_index:
                m.addConstr(
                    gp.quicksum(
                        zsplit[j, k, t]
                        for j in range(self.p)
                        for k in range(len(self._threshold_candidates_[j]))
                    ) == d[t],
                    name=f"one_split_choice[{t}]",
                )

            # link a[j,t] = sum_k zsplit[j,k,t]
            m.addConstrs(
                (
                    a[j, t] == gp.quicksum(zsplit[j, k, t] for k in range(len(self._threshold_candidates_[j])))
                    for j in range(self.p) for t in self.b_index
                ),
                name="a_from_zsplit",
            )

            # set b[t] to chosen threshold (or 0 if inactive)
            for t in self.b_index:
                m.addConstr(
                    b[t]
                    == gp.quicksum(
                        float(self._threshold_candidates_[j][k]) * zsplit[j, k, t]
                        for j in range(self.p)
                        for k in range(len(self._threshold_candidates_[j]))
                    ),
                    name=f"b_from_zsplit[{t}]",
                )

        # -----------------------
        # Residual / abs
        # -----------------------
        for i in range(self.n):
            m.addConstr(
                r[i] == y[i] - gp.quicksum(x[i, j] * beta[j] for j in range(self.p)) - s[i],
                name=f"res_def[{i}]",
            )
            m.addConstr(u[i] >= r[i], name=f"abs_pos[{i}]")
            m.addConstr(u[i] >= -r[i], name=f"abs_neg[{i}]")

        # -----------------------
        # Routing constraints
        # -----------------------
        min_dis = self._calMinDist(x)

        if self.routing_formulation == "indicator":
            # Original indicator routing (kept for compatibility).
            # NOTE: we do NOT use rdir here; instead we keep your old logic if you want it,
            # but indicators require conditioning on "active" too. This is exactly why bigM is faster.
            #
            # If you really want indicator routing, it's better to keep your prior version with rdir.
            #
            # We'll implement a safe indicator version using bigM gating on d[t] via an aux rdir.
            rdir = m.addVars(self.n, self.b_index, vtype=GRB.BINARY, name="rdir")

            for t in self.b_index:
                rt = 2 * t + 1
                for i in range(self.n):
                    m.addConstr(rdir[i, t] <= q[i, rt], name=f"rdir1[{i},{t}]")
                    m.addConstr(rdir[i, t] <= d[t],      name=f"rdir2[{i},{t}]")
                    m.addConstr(rdir[i, t] >= q[i, rt] + d[t] - 1, name=f"rdir3[{i},{t}]")

            for t in self.b_index:
                lt = 2 * t
                for i in range(self.n):
                    expr_left = gp.quicksum(a[j, t] * (x[i, j] + float(min_dis[j])) for j in range(self.p))
                    m.addGenConstrIndicator(q[i, lt], True, expr_left <= b[t], name=f"ind_left[{i},{t}]")

                    expr_right = gp.quicksum(a[j, t] * x[i, j] for j in range(self.p))
                    m.addGenConstrIndicator(rdir[i, t], True, expr_right >= b[t], name=f"ind_right[{i},{t}]")
        else:
            # Faster: pure linear big-M routing, no rdir.
            # Tight global M from feature ranges (safe, usually much faster than many indicators).
            feat_min = self.feature_mins_
            feat_max = self.feature_maxs_
            M_left = float(np.max((feat_max + np.array(min_dis)) - feat_min)) if self.p > 0 else 1.0
            M_right = float(np.max(feat_max - feat_min)) if self.p > 0 else 1.0

            for t in self.b_index:
                lt = 2 * t
                rt = 2 * t + 1
                for i in range(self.n):
                    # If q[i,left]=1 then selected feature value + min_dis <= b[t]
                    expr_left = gp.quicksum(a[j, t] * (x[i, j] + float(min_dis[j])) for j in range(self.p))
                    m.addConstr(expr_left <= b[t] + M_left * (1 - q[i, lt]), name=f"bm_left[{i},{t}]")

                    # If q[i,right]=1 AND d[t]=1 then selected feature value >= b[t]
                    expr_right = gp.quicksum(a[j, t] * x[i, j] for j in range(self.p))
                    # activate only when both are 1: (2 - q_right - d) == 0
                    m.addConstr(
                        expr_right >= b[t] - M_right * (2 - q[i, rt] - d[t]),
                        name=f"bm_right[{i},{t}]",
                    )

        # -----------------------
        # Leaf value selection
        # -----------------------
        if self.leaf_value_formulation == "indicator":
            for lf in self.l_index:
                for i in range(self.n):
                    m.addGenConstrIndicator(q[i, lf], True, (s[i] - gamma[lf] == 0.0), name=f"ind_leafval[{i},{lf}]")
        else:
            # Tight big-M using gamma bounds (removes n*L indicators)
            M_gamma = float(Upper - Lower)
            for i in range(self.n):
                for lf in self.l_index:
                    m.addConstr(s[i] - gamma[lf] <= M_gamma * (1 - q[i, lf]), name=f"bm_leaf1[{i},{lf}]")
                    m.addConstr(gamma[lf] - s[i] <= M_gamma * (1 - q[i, lf]), name=f"bm_leaf2[{i},{lf}]")

        return m, a, b, d, q, l, N, beta, gamma, s, r, u, zsplit

    # ---------------------------
    # Split candidate computation
    # ---------------------------
    def _compute_threshold_candidates(self, x: np.ndarray) -> list[np.ndarray]:
        """
        Candidates for b[t] must be interpretable and consistent with your strict-left encoding.
        Your encoding effectively uses b[t] as the *minimum value on the right branch*,
        so candidates as (unique feature values excluding the minimum) are a good fit.
        """
        cands: list[np.ndarray] = []
        for j in range(x.shape[1]):
            vals = np.unique(x[:, j].astype(float))
            vals = np.sort(vals)
            if vals.size <= 1:
                cands.append(np.array([0.0], dtype=float))
                continue

            # exclude minimum: choosing b=min would make left branch infeasible except empty
            cand = vals[1:].copy()

            if self.max_thresholds is not None and cand.size > self.max_thresholds:
                K = self.max_thresholds
                if self.threshold_strategy == "quantile":
                    qs = np.linspace(0.0, 1.0, K)
                    cand = np.quantile(cand, qs)
                    cand = np.unique(cand)
                else:
                    # uniform index subsample
                    idx = np.linspace(0, cand.size - 1, K).round().astype(int)
                    cand = np.unique(cand[idx])

            if cand.size == 0:
                cand = np.array([vals[-1]], dtype=float)

            cands.append(cand.astype(float))
        return cands

    # ---------------------------
    # Warm start
    # ---------------------------
    def _setStart(self, x, y, a, b, d, q, l, N, beta, gamma, s, r, u, zsplit):
        Lower, Upper = self.gamma_bounds
        min_dis = self._calMinDist(x)

        clf = tree.DecisionTreeRegressor(
            max_depth=self.max_depth,
            min_samples_leaf=self.min_samples_leaf,
            random_state=self.seed,
        )
        clf.fit(x, y)
        rules = self._getRules(clf)

        d0 = {t: 0 for t in self.b_index}
        b0 = {t: 0.0 for t in self.b_index}
        split_feat = {}
        cart_thr = {}

        for t in self.b_index:
            rt = rules[t]
            if rt.feat is None or rt.feat == tree._tree.TREE_UNDEFINED:
                continue
            f = int(rt.feat)
            split_feat[t] = f
            cart_thr[t] = float(rt.threshold)
            d0[t] = 1

        # enforce parent-active
        for t in self.b_index:
            if t != 1 and d0[t] == 1 and d0.get(t // 2, 0) == 0:
                d0[t] = 0
                split_feat.pop(t, None)
                cart_thr.pop(t, None)

        # choose feasible b0 under strict-left encoding
        left_vals = {t: [] for t in split_feat}
        right_vals = {t: [] for t in split_feat}

        for i in range(self.n):
            t = 1
            while t not in self.l_index:
                if d0.get(t, 0) == 0:
                    t = 2 * t + 1
                    continue
                f = split_feat[t]
                thr = cart_thr[t]
                val = x[i, f]
                if val + self.eps_tie >= thr:
                    right_vals[t].append(val)
                    t = 2 * t + 1
                else:
                    left_vals[t].append(val)
                    t = 2 * t

        for t in list(split_feat.keys()):
            f = split_feat[t]
            if len(left_vals[t]) == 0 or len(right_vals[t]) == 0:
                d0[t] = 0
                split_feat.pop(t, None)
                cart_thr.pop(t, None)
                continue
            rmin = float(np.min(right_vals[t]))
            lmax = float(np.max(left_vals[t]))
            if lmax + float(min_dis[f]) > rmin + 1e-12:
                d0[t] = 0
                split_feat.pop(t, None)
                cart_thr.pop(t, None)
                continue
            b0[t] = rmin

        # enforce parent-active again
        for t in self.b_index:
            if t != 1 and d0.get(t, 0) == 1 and d0.get(t // 2, 0) == 0:
                d0[t] = 0
                split_feat.pop(t, None)
                cart_thr.pop(t, None)
                b0[t] = 0.0

        # beta start
        try:
            beta_start = np.linalg.lstsq(x, y, rcond=None)[0]
        except np.linalg.LinAlgError:
            beta_start = np.zeros(self.p, dtype=float)

        if self.beta_bounds is not None:
            lb, ub = self.beta_bounds
            beta_start = np.clip(beta_start, lb, ub)

        residuals = y - x @ beta_start

        def route_leaf(xi):
            t = 1
            while t not in self.l_index:
                if d0.get(t, 0) == 0:
                    t = 2 * t + 1
                    continue
                f = split_feat[t]
                thr = b0[t]
                if xi[f] + self.eps_tie >= thr:
                    t = 2 * t + 1
                else:
                    t = 2 * t
            return t

        assignments = [route_leaf(x[i, :]) for i in range(self.n)]

        leaf_counts = {lf: 0 for lf in self.l_index}
        res_by_leaf = {lf: [] for lf in self.l_index}
        for i, lf in enumerate(assignments):
            leaf_counts[lf] += 1
            res_by_leaf[lf].append(float(residuals[i]))

        if any((cnt > 0 and cnt < self.min_samples_leaf) for cnt in leaf_counts.values()):
            # fallback: single leaf
            self._seed_single_leaf_start(x, y, beta_start, residuals, a, b, d, q, l, N, beta, gamma, s, r, u, zsplit)
            return

        # write starts for splits
        # if using zsplit (discrete thresholds), we must seed those too
        for t in self.b_index:
            if d0.get(t, 0) == 0:
                d[t].Start = 0
                b[t].Start = 0.0
                for f in range(self.p):
                    a[f, t].Start = 0
            else:
                d[t].Start = 1
                fsel = split_feat[t]
                for f in range(self.p):
                    a[f, t].Start = 1 if f == fsel else 0

                # threshold start
                bt = float(b0[t])
                if zsplit is None:
                    b[t].Start = bt
                else:
                    # project bt to nearest candidate >= bt (safe for strict-left encoding)
                    cand = self._threshold_candidates_[fsel]
                    idx = int(np.searchsorted(cand, bt, side="left"))
                    if idx >= len(cand):
                        idx = len(cand) - 1
                    b[t].Start = float(cand[idx])

        # seed zsplit vars if present
        if zsplit is not None:
            # default 0
            for (j, k, t) in zsplit.keys():
                zsplit[j, k, t].Start = 0

            for t in self.b_index:
                if d0.get(t, 0) == 0:
                    continue
                fsel = split_feat[t]
                bt = float(b[t].Start)
                cand = self._threshold_candidates_[fsel]
                ksel = int(np.searchsorted(cand, bt, side="left"))
                ksel = min(ksel, len(cand) - 1)
                zsplit[fsel, ksel, t].Start = 1

        for j in range(self.p):
            beta[j].Start = float(beta_start[j])

        gamma0 = {}
        for lf in self.l_index:
            g = float(np.median(res_by_leaf[lf])) if leaf_counts[lf] > 0 else 0.0
            g = float(np.clip(g, Lower, Upper))
            gamma[lf].Start = g
            gamma0[lf] = g
            l[lf].Start = 1 if leaf_counts[lf] > 0 else 0
            N[lf].Start = float(leaf_counts[lf])

        # q starts
        for i in range(self.n):
            for t in self.n_index:
                q[i, t].Start = 0
            q[i, 1].Start = 1

            t = 1
            while t not in self.l_index:
                if d0.get(t, 0) == 0:
                    rt = 2 * t + 1
                    q[i, rt].Start = 1
                    t = rt
                    continue

                f = split_feat[t]
                thr = float(b[t].Start)
                if x[i, f] + self.eps_tie >= thr:
                    rt = 2 * t + 1
                    q[i, rt].Start = 1
                    t = rt
                else:
                    lt = 2 * t
                    q[i, lt].Start = 1
                    t = lt

            lf = t
            sval = gamma0[lf]
            s[i].Start = sval

            ri = float(residuals[i] - sval)
            r[i].Start = ri
            u[i].Start = abs(ri)

    def _seed_single_leaf_start(self, x, y, beta_start, residuals, a, b, d, q, l, N, beta, gamma, s, r, u, zsplit):
        Lower, Upper = self.gamma_bounds
        default_leaf = self.l_index[-1]
        gval = float(np.clip(np.median(residuals), Lower, Upper)) if self.n > 0 else 0.0

        for t in self.b_index:
            d[t].Start = 0
            b[t].Start = 0.0
            for f in range(self.p):
                a[f, t].Start = 0

        if zsplit is not None:
            for key in zsplit.keys():
                zsplit[key].Start = 0

        for j in range(self.p):
            beta[j].Start = float(beta_start[j])

        for lf in self.l_index:
            l[lf].Start = 1 if lf == default_leaf else 0
            N[lf].Start = float(self.n) if lf == default_leaf else 0.0
            gamma[lf].Start = gval if lf == default_leaf else 0.0

        for i in range(self.n):
            for t in self.n_index:
                q[i, t].Start = 0
            q[i, 1].Start = 1

            t = 1
            while t not in self.l_index:
                t = 2 * t + 1
                q[i, t].Start = 1

            s[i].Start = gval
            ri = float(residuals[i] - gval)
            r[i].Start = ri
            u[i].Start = abs(ri)

    # ---------------------------
    # Fast polish step
    # ---------------------------
    def _polish_solution(self, m: gp.Model, *, a, d, q, l, zsplit):
        """
        Fix binaries to incumbent (rounded) and re-optimize continuous vars as LP.
        This is typically very fast and can noticeably improve objective quality
        when the MIP is stopped early.
        """
        # Collect binaries
        binaries = []
        for t in self.b_index:
            binaries.append(d[t])
            for j in range(self.p):
                binaries.append(a[j, t])
        for i in range(self.n):
            for t in self.n_index:
                binaries.append(q[i, t])
        for lf in self.l_index:
            binaries.append(l[lf])
        if zsplit is not None:
            for key in zsplit.keys():
                binaries.append(zsplit[key])

        # Fix them
        for v in binaries:
            val = 1.0 if v.X >= 0.5 else 0.0
            v.LB = val
            v.UB = val

        m.update()

        # Re-optimize (LP)
        old_tl = m.Params.TimeLimit
        m.Params.TimeLimit = max(0.0, float(self.polish_timelimit))
        m.optimize()
        m.Params.TimeLimit = old_tl

    # ---------------------------
    # Utilities
    # ---------------------------
    @staticmethod
    def _calMinDist(x: np.ndarray):
        min_dis = []
        for j in range(x.shape[1]):
            xj = np.unique(x[:, j])
            if len(xj) <= 1:
                min_dis.append(1.0)
                continue
            xj = np.sort(xj)
            diffs = np.diff(xj)
            diffs = diffs[diffs > 0]
            min_dis.append(float(np.min(diffs)) if diffs.size else 1.0)
        return min_dis

    def _getRules(self, clf):
        node_map = {1: 0}
        for t in self.b_index:
            node_map[2 * t] = -1
            node_map[2 * t + 1] = -1
            if node_map[t] == -1:
                continue
            node_map[2 * t] = clf.tree_.children_left[node_map[t]]
            node_map[2 * t + 1] = clf.tree_.children_right[node_map[t]]

        rule = namedtuple("Rules", ("feat", "threshold", "value"))
        rules = {}
        for t in self.b_index:
            i = node_map[t]
            rules[t] = rule(None, None, None) if i == -1 else rule(
                clf.tree_.feature[i], clf.tree_.threshold[i], clf.tree_.value[i, 0]
            )
        for t in self.l_index:
            i = node_map[t]
            rules[t] = rule(None, None, None) if i == -1 else rule(None, None, clf.tree_.value[i, 0])
        return rules


In [ ]:
#!/usr/bin/env python3
# benchmark_iorfa_real.py

import time
import warnings
from dataclasses import dataclass
from typing import Callable, Dict, List, Optional, Tuple

import numpy as np

# --- Baselines + utilities ---
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_california_housing, load_diabetes, fetch_openml
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor


@dataclass
class DatasetSpec:
    name: str
    loader: Callable[[int], Tuple[object, np.ndarray]]  # (seed) -> (X (DataFrame-like), y)


def _onehot_dense():
    """
    OneHotEncoder that yields dense output across sklearn versions.
    """
    try:
        return OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    except TypeError:
        return OneHotEncoder(handle_unknown="ignore", sparse=False)


def make_preprocessor_for_df(X):
    """
    Builds a ColumnTransformer for numeric + categorical columns.
    Outputs:
      - sparse for baseline pipelines (fine)
      - dense for IORFA (we'll set sparse_threshold=0.0 to force dense if possible)
    """
    # Works for pandas DataFrame; if user passes numpy, treat all numeric.
    try:
        import pandas as pd  # noqa: F401
        is_df = hasattr(X, "dtypes")
    except Exception:
        is_df = hasattr(X, "dtypes")

    if not is_df:
        # All numeric
        numeric_pipe = Pipeline(
            steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
        )
        return ColumnTransformer([("num", numeric_pipe, slice(0, X.shape[1]))], remainder="drop")

    num_cols = list(X.select_dtypes(include=["number", "bool"]).columns)
    cat_cols = [c for c in X.columns if c not in num_cols]

    numeric_pipe = Pipeline(
        steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
    )
    cat_pipe = Pipeline(
        steps=[("imputer", SimpleImputer(strategy="most_frequent")), ("onehot", _onehot_dense())]
    )

    return ColumnTransformer(
        transformers=[
            ("num", numeric_pipe, num_cols),
            ("cat", cat_pipe, cat_cols),
        ],
        remainder="drop",
        sparse_threshold=0.0,  # try to keep output dense (helps IORFA)
    )


def subsample(X, y, max_n: Optional[int], seed: int):
    if max_n is None:
        return X, y
    n = len(y)
    if n <= max_n:
        return X, y
    rng = np.random.default_rng(seed)
    idx = rng.choice(n, size=max_n, replace=False)
    if hasattr(X, "iloc"):
        return X.iloc[idx], y[idx]
    return X[idx], y[idx]


def metrics(y_true, y_pred):
    mae = float(mean_absolute_error(y_true, y_pred))
    mse = float(mean_squared_error(y_true, y_pred))
    rmse = float(np.sqrt(mse))
    r2 = float(r2_score(y_true, y_pred))
    return {"MAE": mae, "RMSE": rmse, "R2": r2, "MSE": mse}



def make_datasets() -> List[DatasetSpec]:
    def diabetes(seed: int):
        d = load_diabetes()
        X = d.data
        y = d.target
        rng = np.random.default_rng(seed)
        idx = rng.permutation(len(y))
        return X[idx], y[idx]

    def california(seed: int):
        d = fetch_california_housing()
        X = d.data
        y = d.target
        rng = np.random.default_rng(seed)
        idx = rng.permutation(len(y))
        return X[idx], y[idx]

    def openml_regression(data_id: int, target_col: Optional[str] = None):
        def _loader(seed: int):
            bunch = fetch_openml(data_id=data_id, as_frame=True)
            X = bunch.data
            y = bunch.target
            # If multi-target, pick one column
            if hasattr(y, "shape") and len(getattr(y, "shape", ())) == 2 and y.shape[1] > 1:
                if target_col is not None and hasattr(y, "__getitem__"):
                    yv = np.asarray(y[target_col], dtype=float).ravel()
                else:
                    yv = np.asarray(y.iloc[:, 0], dtype=float).ravel()
            else:
                yv = np.asarray(y, dtype=float).ravel()

            rng = np.random.default_rng(seed)
            idx = rng.permutation(len(yv))
            return X.iloc[idx], yv[idx]
        return _loader

    return [
        DatasetSpec("diabetes_sklearn", diabetes),
        DatasetSpec("california_housing_sklearn", california),

        # OpenML real regression datasets (by data_id)
        DatasetSpec("abalone_183", openml_regression(183)),
        DatasetSpec("wine_quality_287", openml_regression(287)),
        DatasetSpec("kin8nm_189", openml_regression(189)),
        DatasetSpec("cpu_act_197", openml_regression(197)),
        DatasetSpec("cpu_small_562", openml_regression(562)),
        DatasetSpec("yacht_hydrodynamics_42370", openml_regression(42370)),

        # Energy Efficiency has 2 targets; pick one if present (names vary; fallback is first col)
        DatasetSpec("energy_efficiency_1472", openml_regression(1472, target_col="Y1")),

        DatasetSpec("concrete_strength_44959", openml_regression(44959)),
    ]


def make_baselines(seed: int) -> Dict[str, object]:
    return {
        "LinearRegression": LinearRegression(),
        "Ridge(alpha=1.0)": Ridge(alpha=1.0, random_state=seed),
        "Lasso(alpha=0.01)": Lasso(alpha=0.01, max_iter=20000, random_state=seed),
        "CART": DecisionTreeRegressor(random_state=seed),
        "RandomForest": RandomForestRegressor(n_estimators=300, random_state=seed, n_jobs=-1),
        "ExtraTrees": ExtraTreesRegressor(n_estimators=400, random_state=seed, n_jobs=-1),
        "GradientBoosting": GradientBoostingRegressor(random_state=seed),
    }


def main():
    warnings.filterwarnings("ignore")

    # ---- knobs ----
    seed = 0
    test_size = 0.25

    # Keep datasets "not too big" in practice
    max_n_all = 30        # subsample for ALL models (set None to use full)

    # IORFA settings (safe-ish defaults)
    iorfa_max_depth = 6
    iorfa_alpha = 0.01
    iorfa_timelimit = 1200    # seconds per dataset
    iorfa_output = True

    datasets = make_datasets()
    baselines = make_baselines(seed)

    rows = []

    for ds in datasets:
        print(f"\n=== {ds.name} ===")
        try:
            X, y = ds.loader(seed)
        except Exception as e:
            print(f"SKIP (load failed): {type(e).__name__}: {e}")
            continue

        # subsample globally (keeps “not too big”)
        X, y = subsample(X, y, max_n_all, seed)

        # split
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=seed
        )

        # preprocessing
        pre = make_preprocessor_for_df(X_train)

        # ----- baselines -----
        for name, model in baselines.items():
            pipe = Pipeline([("pre", pre), ("model", model)])
            t0 = time.perf_counter()
            pipe.fit(X_train, y_train)
            fit_s = time.perf_counter() - t0
            pred = pipe.predict(X_test)
            m = metrics(y_test, pred)
            rows.append({
                "dataset": ds.name, "model": name,
                "n_train": len(y_train), "n_test": len(y_test),
                "RMSE": m["RMSE"], "MAE": m["MAE"], "R2": m["R2"],
                "fit_seconds": fit_s, "iorfa_optgap": None, "iorfa_status": None
            })
            print(f"{name:16s} MAE={m['MAE']:.4f}  RMSE={m['RMSE']:.4f}  R2={m['R2']:.4f}  fit_s={fit_s:.2f}")


        # ----- IORFA -----
        try:
            # (no subsampling needed since max_n_all=max_n_iorfa)
            Xi_train, yi_train = X_train, y_train
            Xi_test,  yi_test  = X_test,  y_test

            pre_i = make_preprocessor_for_df(Xi_train)
            Xi_train_p = pre_i.fit_transform(Xi_train)
            Xi_test_p  = pre_i.transform(Xi_test)

            # scale y for IORFA only
            y_mu = float(np.mean(yi_train))
            y_sd = float(np.std(yi_train)) if float(np.std(yi_train)) > 1e-9 else 1.0
            yi_train_s = (yi_train - y_mu) / y_sd

            # tighter gamma bounds in scaled space
            gamma_bounds = (-5.0, 5.0)

            mss = max(2, min(10, int(0.05 * len(yi_train_s))))

            model = OptimalRegressionTreeFlow(
    max_depth=2,
    min_samples_leaf=None,
    alpha=1e-3,
    routing_formulation="bigm",
    leaf_value_formulation="bigm",
    max_thresholds=32,            # try 16/32/64
    lambda_beta_l1=0,          # sparsify linear term (RuleFit-ish)
    lambda_gamma_l1=0.0,          # optional
    gamma_bounds=(-5.0, 5.0),  # IMPORTANT
    warmstart=False,           # until fixed
    polish=True,
    polish_timelimit=1.0,
    timelimit=600,
    output=True,
)

            t0 = time.perf_counter()
            model.fit(Xi_train_p, yi_train_s)
            fit_s = time.perf_counter() - t0

            pred_s = model.predict(Xi_test_p)
            pred   = pred_s * y_sd + y_mu  # unscale back to original y

            m = metrics(yi_test, pred)
            optgap = getattr(model, "optgap", None)
            status = getattr(getattr(model, "m", None), "status", None)

            print(f"{'IORFA':16s} MAE={m['MAE']:.4f}  RMSE={m['RMSE']:.4f}  R2={m['R2']:.4f}  fit_s={fit_s:.2f}  gap={optgap}  status={status}")


        except Exception as e:
            print(f"IORFA failed: {type(e).__name__}: {e}")
            rows.append({
                "dataset": ds.name,
                "model": f"IORFA(d={iorfa_max_depth},a={iorfa_alpha})",
                "n_train": None, "n_test": None,
                "RMSE": None, "MAE": None, "R2": None,
                "fit_seconds": None, "iorfa_optgap": None, "iorfa_status": None,
                "error": f"{type(e).__name__}: {e}",
            })

    # ----- save results -----
    try:
        import pandas as pd
        df = pd.DataFrame(rows)
        df.to_csv("benchmark_results_real.csv", index=False)
        print("\nSaved: benchmark_results_real.csv")
        print("\nTop rows (sorted by dataset then RMSE):")
        print(df.sort_values(["dataset", "RMSE"], na_position="last").head(30).to_string(index=False))
    except ImportError:
        print("\nInstall pandas to save CSV nicely: pip install pandas")


if __name__ == "__main__":
    main()



=== diabetes_sklearn ===
LinearRegression RMSE=38.0990  MAE=28.5090  R2=0.6989  fit_s=0.01
Ridge(alpha=1.0) RMSE=39.4934  MAE=31.4610  R2=0.6764  fit_s=0.00
Lasso(alpha=0.01) RMSE=38.1176  MAE=28.4796  R2=0.6986  fit_s=0.00
CART             RMSE=64.4963  MAE=58.8462  R2=0.1370  fit_s=0.00
RandomForest     RMSE=44.0890  MAE=38.8672  R2=0.5967  fit_s=0.30
ExtraTrees       RMSE=48.1208  MAE=40.9398  R2=0.5196  fit_s=0.24
GradientBoosting RMSE=50.8034  MAE=45.3818  R2=0.4645  fit_s=0.02
Training data include 37 instances, 10 features.
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2755804
Academic license 2755804 - for non-commercial use only - registered to pa___@mit.edu
Set parameter OutputFlag to value 1
Set parameter LogToConsole to value 1
Set parameter TimeLimit to value 300
Set parameter Threads to value 0
Set parameter Seed to value 0
Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (armlinux64 - "Ubuntu 24.04.2 LTS")

CPU model: ARM64
Thread co